In [1]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [2]:
# Данный блок в области 1 выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
# запрещается скрывать предупреждения системы
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
# pip install

In [3]:
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

### Область работы 2 (выполнение лучшей модели)

In [4]:
# Данный блок(и) в области 2 выполняется преподавателем
#
# В области находится одна, единственная, итоговая модель машинного обучения с однозначными, 
# зафиксированными параметрами
#
# В данной области категорически запрещается искать, выбирать, улучшать, оптимизировать, 
# тюниговать и т.д. модель машинного обучения

In [5]:
# Путь к тренировочному набору
path_train = 'train.csv' # содержит только имя файла, без имен папок
# Путь к тестовому набору
path_test  = 'test.csv' # содержит только имя файла, без имен папок

In [6]:
# Блок(и) обучения и поверки модели

In [7]:
df_train = pd.read_csv(path_train)
df_test = pd.read_csv(path_test)

In [8]:
df_train['quality'] = df_train.quality.apply(lambda x: 1 if x>=7 else 0)
#df_train.head()

In [9]:
#Удаление выбросов
set_index_for_drop = set()
for column in df_train.columns[:-1]:
    Q1 = df_train[column].quantile(0.25)
    Q3 = df_train[column].quantile(0.75)
    IQR = Q3 - Q1
    indexes = list(df_train[(df_train[column] < Q1-1.5*IQR ) | (df_train[column] > Q3+1.5*IQR)].index)
    set_index_for_drop.update(indexes)

df_not_outliers = df_train.drop(index=set_index_for_drop).copy()

In [10]:
#Разделяем на X и y
X, y = df_not_outliers.drop(columns='quality'), df_not_outliers['quality']

In [11]:
params = {
    'penalty': 'elasticnet', 
    'solver': 'saga', 
    'C':0.025, 
    'class_weight': None,  
    'max_iter': 250,  
    'l1_ratio': 0.8,   
    'multi_class': 'multinomial'  
}

pipe = Pipeline([('preprocessing', MinMaxScaler()), 
                 ('clf',           LogisticRegression(**params))])

pipe.fit(X, y)  # т.к. модель уже подобрана обучаемся в конце всегда на всем наборе имеющихся у нас данных

Pipeline(steps=[('preprocessing', MinMaxScaler()),
                ('clf',
                 LogisticRegression(C=0.025, l1_ratio=0.8, max_iter=250,
                                    multi_class='multinomial',
                                    penalty='elasticnet', solver='saga'))])

In [12]:
# Блок предсказания с использованием тестового набора

In [13]:
final_pred = pipe.predict(df_test)
#final_pred

In [14]:
# Название вектора предсказанных значений  y_predict полученого на основании тестового набора
y_predict = final_pred